In [2]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 4.5 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [18]:
from pl_dataloader import dataset_loader
from config import Config
import pandas as pd
from skmultilearn.model_selection import iterative_train_test_split as itts
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [21]:
config = Config('/pvc/ContinualClinicalNLU/pipeline_test/config.yml')
dataset = dataset_loader(config)

In [55]:
achepa_codiesp_train = dataset.frames['achepa_codiesp_train']

In [77]:
achepa_codiesp_train['labels'][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0.

In [23]:
dataset_names = list(dataset.frames.keys())
# list(dataset_names)
dataset_names.sort()
dataset_names

['achepa_codiesp_dev',
 'achepa_codiesp_test',
 'achepa_codiesp_train',
 'codiesp_CCS_dev',
 'codiesp_CCS_test',
 'codiesp_CCS_train',
 'mimic_codiesp_dev',
 'mimic_codiesp_test',
 'mimic_codiesp_train']

In [24]:
lengths = [len(dataset.frames[dataset_names[0]]), len(dataset.frames[dataset_names[1]]), len(dataset.frames[dataset_names[2]])]
print(lengths[2] / sum(lengths))

lengths = [len(dataset.frames[dataset_names[3]]), len(dataset.frames[dataset_names[4]]), len(dataset.frames[dataset_names[5]])]
print(lengths[0] / sum(lengths))

lengths = [len(dataset.frames[dataset_names[6]]), len(dataset.frames[dataset_names[7]]), len(dataset.frames[dataset_names[8]])]
print(lengths[0] / sum(lengths))


0.6682027649769585
0.1827309236947791
0.16448945511352925


In [25]:
achepa_codiesp = pd.concat([dataset.frames[dataset_names[0]], dataset.frames[dataset_names[1]], dataset.frames[dataset_names[2]]]).to_numpy()
codiesp_CCS = pd.concat([dataset.frames[dataset_names[3]], dataset.frames[dataset_names[4]], dataset.frames[dataset_names[5]]]).to_numpy()
mimic_codiesp = pd.concat([dataset.frames[dataset_names[6]], dataset.frames[dataset_names[7]], dataset.frames[dataset_names[8]]]).to_numpy()

In [26]:
arrays = [achepa_codiesp, codiesp_CCS, mimic_codiesp]

In [134]:
from skmultilearn.model_selection import iterative_train_test_split

def split_and_transform(frame):
    # Your features (column 1)
    X = frame[:, 0:1]

    # Convert the label lists to a 2D array (column 2)
    y = np.vstack(frame[:,1])

    # Split the data
    X_temp, y_temp, X_train, y_train = iterative_train_test_split(X, y, test_size=0.66)
    print(f'x_temp.shape:{X_temp.shape}, y_temp.shape: {y_temp.shape}')
    print(f'x_train.shape:{X_train.shape} y_train.shape: {y_train.shape}')
    X_dev, y_dev, X_test, y_test = iterative_train_test_split(X_temp, y_temp, test_size=0.5)

    return (X_train, y_train), (X_dev, y_dev), (X_test, y_test)

train, dev, test = split_and_transform(achepa_codiesp)

x_temp.shape:(806, 1), y_temp.shape: (806, 334)
x_train.shape:(1581, 1) y_train.shape: (1581, 334)


In [135]:
def back_to_csv(split_set, path, name):
    reshaped_array = np.empty((split_set[1].shape[0], 1), dtype=object)
    for i in range(split_set[1].shape[0]):
        reshaped_array[i, 0] = split_set[1][i, :]
    frame = pd.DataFrame(np.concatenate((split_set[0],reshaped_array), axis=1))
    frame = frame.rename(columns={0:'text',1:'labels'})
    frame['labels'] = frame['labels'].apply(list)
    full_path = path + '/' + name + '.csv'
    frame.to_csv(full_path, index=False)
    return

In [136]:
import os
path = '/pvc/data/continualLearning/stratified/'
if not os.path.exists(path):
     os.makedirs(path)
for idx, i in enumerate(arrays):
    train, dev, test = split_and_transform(i)
    splits = [dev,test,train]
    if idx != 0:
            idx = idx*3
    for idx2, split in enumerate(splits):
        index = idx + idx2
        print(index)
        back_to_csv(split,path,dataset_names[index])

x_temp.shape:(806, 1), y_temp.shape: (806, 334)
x_train.shape:(1581, 1) y_train.shape: (1581, 334)
0
1
2
x_temp.shape:(356, 1), y_temp.shape: (356, 334)
x_train.shape:(640, 1) y_train.shape: (640, 334)
3
4
5
x_temp.shape:(12776, 1), y_temp.shape: (12776, 334)
x_train.shape:(24351, 1) y_train.shape: (24351, 334)
6
7
8


In [2]:
import pandas as pd
from transformers import AutoTokenizer
import numpy as np
import os
import glob
from typing import Tuple
from torch.utils.data import Dataset as torch_Dataset, DataLoader
from config import Config


/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [73]:
frame2 =  pd.read_csv('/pvc/test.csv', converters={'label': eval})
frame2['label'] =frame2['label'].astype(np.array)

In [99]:
test = pd.read_csv('/pvc/data/continualLearning/stratified/achepa_codiesp_dev.csv', converters={'label': eval})
test

,text,label
0,γενος: αρρεν διαγνωση εξοδου: προκαρδιο αλγος....,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,γενος: θηλυ διαγνωση εξοδου:οξυ εμφραγμα του μ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,γενος: θηλυ διαγνωση εξοδου: λιποθυμικο επεισο...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,γενος: αρρεν διαγνωση εξοδου: μικρη περικαρδια...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,γενος: αρρεν διαγνωση εξοδου: ατυπο θωρακικο α...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
389,γενος: αρρεν διαγνωση εξοδου: στενωση καρωτιδο...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
390,γενος: θηλυ διαγνωση εξοδου:προκαρδιο αλγος. π...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
391,γενος: θηλυ διαγνωση εξοδου: ατυπο θωρακικο αλ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
392,γενος: αρρεν διαγνωση εξοδου: ατυπο προκαρδιο ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [130]:
class dataset_loader_stratified:   
    def __init__(self, config):
        self.path = config.dataset.path
        self.num_entries = config.dataset.num_entries
        self.specific_sets = config.train.languages
        self.frames = self.get_dataset_dict()
        self.num_labels = self.get_num_labels()

    def load_single_df(self, file: str) -> pd.DataFrame:
        """ Load a single .csv file, transform the labels from text into nummeric, one-hot encoded version"""

        converter = {'label': eval}
        df = pd.read_csv(file, converters = converter)
        df.rename(columns={'TEXT': 'text', 'notes': 'text'}, inplace=True)
        df['label'] = df['label'].apply(np.array)
        return df

    def load_multiple_dfs(self) -> dict:
        """ Load all .csv files in a given directory, using the load_and_label_df function for each file by default, 
        return a dictionary with the dataframe, and a dictionary mapping the nummeric labels to the text labels
        specific_sets: str, default None, could be 'mimic' or 'codiesp' to load only one set of dataset"""
        
        # get all files in the directory
        all_files = glob.glob(os.path.join(self.path, "*.csv"))

        # filter the files if specific_sets is given
        for i in self.specific_sets:
            if not any(i in file for file in all_files):
                raise ValueError(f'{i} is not found in the data directory!')
            
        if self.specific_sets:
            all_files = [i for j in self.specific_sets for i in all_files if j in i]

        file_names = [i.split('/')[-1].split('.')[0] for i in all_files]  # get the file name without the path to use as dictionary key
        print(file_names)
        # load all files into the dictionaries
        frames = {}
        for idx, i in enumerate(file_names):
            j = i.split('_')
            i = f'{j[0]}_{j[1]}_{j[-1]}'
            if self.num_entries != 'all':
                frames[i] = self.load_single_df(all_files[idx]).head(self.num_entries)
            else:
                frames[i] = self.load_single_df(all_files[idx])
            
        return frames


    def get_dataset_dict(self) -> pd.DataFrame:
        """ Create a dictionary of datasets from a dictionary of dataframes"""

        frames = self.load_multiple_dfs()

        return frames
    
    def get_num_labels(self) -> int:
        ''' len(labels) to retrieve the amount of labels'''
        keys = list(self.frames.keys())
        return len(self.frames[keys[0]]['label'][0])

In [131]:
config = Config('/pvc/ContinualClinicalNLU/pipeline_test/config.yml')
dataset = dataset_loader_stratified(config)

['achepa_codiesp_train', 'achepa_codiesp_test', 'achepa_codiesp_dev', 'codiesp_CCS_test', 'codiesp_CCS_dev', 'codiesp_CCS_train', 'mimic_codiesp_dev', 'mimic_codiesp_test', 'mimic_codiesp_train']


In [212]:
keys = list(dataset.frames.keys())
keys.sort()
keys

['achepa_codiesp_dev',
 'achepa_codiesp_test',
 'achepa_codiesp_train',
 'codiesp_CCS_dev',
 'codiesp_CCS_test',
 'codiesp_CCS_train',
 'mimic_codiesp_dev',
 'mimic_codiesp_test',
 'mimic_codiesp_train']

In [226]:
test = dataset.frames[keys[3]].iloc[:,1]

In [224]:
for i in dataset.frames[keys[3]].iloc[:,1]:
    print(i)
    break

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [225]:
for idx, i in enumerate(keys):
    print(i)
    current_frame = dataset.frames[keys[idx]].iloc[:,1]
    df = pd.DataFrame(current_frame.tolist())
    df = df.loc[:, (df==0).all(axis=0)]
    df.columns()
    break

achepa_codiesp_dev


TypeError: 'Int64Index' object is not callable

In [244]:
df = pd.DataFrame(test.tolist())
df

,0,1,2,3,4,5,6,7,8,9,...,324,325,326,327,328,329,330,331,332,333
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [245]:
df = df.loc[:, (df == 0).all(axis=0)]
df

,0,5,6,8,9,13,20,23,24,28,...,272,273,279,286,291,292,312,314,317,333
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [209]:
for i in df[5]:
    if i != 0:
        print(i)


In [231]:
(df != 0).any()

0      False
1       True
2       True
3       True
4       True
       ...  
329     True
330     True
331     True
332     True
333    False
Length: 334, dtype: bool